In [2]:
import torch
from itertools import chain
import pyro.distributions as dist
import pyro
from pyro import poutine
import numpy as np
from itertools import chain
import pdb
from pmextract import extract

In [3]:
from sherlog.program import loads
from sherlog.inference import minibatch, FunctionalEmbedding, Optimizer
from sherlog.interface import print, initialize, minotaur

In [4]:
initialize(port=8014, instrumentation=None)

In [5]:
embedder = FunctionalEmbedding(evidence=lambda s: s)

In [55]:
NTOPICS = 2
NJUDGES = 3
JUDGES_PER_PAPER = 2
NDOCS = 5

In [56]:
topics = torch.randint(0, NTOPICS, (NDOCS,))

In [57]:
judges = torch.tensor(np.array([
    np.random.choice(NJUDGES, JUDGES_PER_PAPER, replace=False) for _ in range(NDOCS)
])).long()

In [58]:
judges.shape

torch.Size([5, 2])

In [59]:
topics.shape

torch.Size([5])

In [60]:
def model():
    yesno = ["yes", "no"]
    with pyro.plate("judges", NJUDGES):
        threshold = pyro.sample("threshold", dist.Normal(4, 0.5))
        with pyro.plate("topics", NTOPICS):
            expertise = pyro.sample("expertise", dist.Gamma(1., 1.))
    with pyro.plate("papers", NDOCS) as papers:
        with poutine.block():
            quality = pyro.sample("quality", dist.Normal(2.5, 1.0))
        with pyro.plate("paperjudge", JUDGES_PER_PAPER) as paperjudge:
            paper_expertise = torch.gather(expertise[topics, :], 0, judges.long())
            score_dist = dist.Normal(quality.float(), paper_expertise.T)
            meetsThresh = pyro.sample("meetsThresh", dist.Bernoulli(probs = 1 - score_dist.cdf(threshold[judges].T))).long()
            obs = [f"meets_thresh({p}, {judges[p,j]}, {yesno[meetsThresh[j, p]]})" for p in papers for j in paperjudge]
    
    params = [f"!parameter paperQuality{i} : real" for i in range(NDOCS)]
    params.extend([f"!parameter judgeThreshold{i} : real" for i in range(NJUDGES)])
    params.extend([f"!parameter judgeExpertise{j}_{t} : real" for j in range(NJUDGES) for t in range(NTOPICS)])
    data = list(chain.from_iterable((f"judged({j}, {p})" for (p,j) in enumerate(judges[:,i])) for i in range(JUDGES_PER_PAPER)))
    data.extend([f"hasTopic({p}, {t})" for p, t in enumerate(topics)])
    data.extend([f"judge_expertise({j}, {t}, judgeExpertise{j}_{t})" for j in range(NJUDGES) for t in range(NTOPICS)])
    data.extend([f"paper_quality({p}, paperQuality{p})" for p in range(NDOCS)])
    data.extend([f"judge_threshold({j}, judgeThreshold{j})" for j in range(NJUDGES)])

    return obs, params + data

In [61]:
obs, data = model()

In [62]:
PROG = """
meet_thresh_prob(P, J; gaussian_survival[TH, Q, E]) <- paper_quality(P, Q),
  judge_expertise(J, T, E), hasTopic(P, T), judged(J, P), judge_threshold(J, TH).
  
meets_thresh(P, J; {yes, no} <~ bernoulli[PROB]) <- meet_thresh_prob(P, J, PROB).
"""

In [63]:
SOURCE = ".\n".join(data) + "." + PROG

In [64]:
print(obs)

[
    'meets_thresh(0, 0, yes)',
    'meets_thresh(0, 2, yes)',
    'meets_thresh(1, 1, yes)',
    'meets_thresh(1, 2, yes)',
    'meets_thresh(2, 0, yes)',
    'meets_thresh(2, 1, yes)',
    'meets_thresh(3, 0, yes)',
    'meets_thresh(3, 2, yes)',
    'meets_thresh(4, 0, yes)',
    'meets_thresh(4, 1, yes)'
]

In [65]:
print(SOURCE)

!parameter paperQuality0 : real.
!parameter paperQuality1 : real.
!parameter paperQuality2 : real.
!parameter paperQuality3 : real.
!parameter paperQuality4 : real.
!parameter judgeThreshold0 : real.
!parameter judgeThreshold1 : real.
!parameter judgeThreshold2 : real.
!parameter judgeExpertise0_0 : real.
!parameter judgeExpertise0_1 : real.
!parameter judgeExpertise1_0 : real.
!parameter judgeExpertise1_1 : real.
!parameter judgeExpertise2_0 : real.
!parameter judgeExpertise2_1 : real.
judged(0, 0).
judged(1, 1).
judged(0, 2).
judged(0, 3).
judged(0, 4).
judged(2, 0).
judged(2, 1).
judged(1, 2).
judged(2, 3).
judged(1, 4).
hasTopic(0, 0).
hasTopic(1, 1).
hasTopic(2, 0).
hasTopic(3, 1).
hasTopic(4, 1).
judge_expertise(0, 0, judgeExpertise0_0).
judge_expertise(0, 1, judgeExpertise0_1).
judge_expertise(1, 0, judgeExpertise1_0).
judge_expertise(1, 1, judgeExpertise1_1).
judge_expertise(2, 0, judgeExpertise2_0).
judge_expertise(2, 1, judgeExpertise2_1).
paper_quality(0, paperQuality0).
paper_quality(1, paperQuality1).
paper_quality(2, paperQuality2).
paper_quality(3, paperQuality3).
paper_quality(4, paperQuality4).
judge_threshold(0, judgeThreshold0).
judge_threshold(1, judgeThreshold1).
judge_threshold(2, judgeThreshold2).
meet_thresh_prob(P, J; gaussian_survival[TH, Q, E]) <- paper_quality(P, Q),
  judge_expertise(J, T, E), hasTopic(P, T), judged(J, P), judge_threshold(J, TH).
  
meets_thresh(P, J; {yes, no} <~ bernoulli[PROB]) <- meet_thresh_prob(P, J, PROB).

In [66]:
program, _ = loads(SOURCE)

In [67]:
optimizer = Optimizer(program, learning_rate=1e-2, samples=10, force=True, cache=False)

In [68]:
optimizer.maximize(*embedder.embed_all(obs))

In [69]:
for batch in minibatch(obs, 10, epochs=10):
    optimizer.maximize(*embedder.embed_all(batch.data))
    print(optimizer.optimize().item())

13.862943649291992

2.823244571685791

2.025430679321289

1.4499273300170898

1.036405324935913

0.7431535720825195

0.5381296873092651

0.39610055088996887

0.29776012897491455

0.22906097769737244